In [2]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch
from pytorch_lightning import Trainer
import tensorboard

## ネットワーク定義

In [3]:
class Net(pl.LightningModule):

    def __init__(self):
        super().__init__()

        self.bn = nn.BatchNorm1d(768)
        self.fc1 = nn.Linear(768, 400)
        self.fc2 = nn.Linear(400, 10)

    def forward(self, x):
        h = self.bn(x)
        h = self.fc1(h)
        h = F.relu(h)
        h = self.fc2(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer

## データセット読み込み

## ネットワークの学習

In [ ]:
# GPUを含めた乱数シード固定
pl.seed_everything(0)
net = Net()

# 学習を行う Trainer
trainer = Trainer(max_epochs=10)

# 学習の実行
trainer.fit(net, train_loader, val_loader)